In [1]:
import pandas as pd
import numpy as np
import matplotlib .pyplot as plt
%matplotlib inline
import quandl


In [92]:
# File to Load (Remember to Change These)
CCI_data_to_load = "startdata/CCI_Raw_data.csv"
BCI_data_to_load = 'startdata/BCI_Raw_data.csv'
CLI_data_to_load = 'startdata/CLI_Raw_data.csv'
# Read School and Student Data File and store into Pandas DataFrames
CCI_data = pd.read_csv(CCI_data_to_load)
BCI_data = pd.read_csv(BCI_data_to_load)
CLI_data = pd.read_csv(CLI_data_to_load)
CLI_data.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,GBR,CLI,AMPLITUD,LTRENDIDX,M,1957-12,97.69984,NaN
1,GBR,CLI,AMPLITUD,LTRENDIDX,M,1958-01,97.73627,NaN
2,GBR,CLI,AMPLITUD,LTRENDIDX,M,1958-02,97.84341,NaN
3,GBR,CLI,AMPLITUD,LTRENDIDX,M,1958-03,97.99134,NaN
4,GBR,CLI,AMPLITUD,LTRENDIDX,M,1958-04,98.15497,NaN


In [93]:
BCI_data.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,ZAF,BCI,AMPLITUD,LTRENDIDX,M,1974-06,102.0001,NaN
1,ZAF,BCI,AMPLITUD,LTRENDIDX,M,1974-07,101.8869,NaN
2,ZAF,BCI,AMPLITUD,LTRENDIDX,M,1974-08,101.7853,NaN
3,ZAF,BCI,AMPLITUD,LTRENDIDX,M,1974-09,101.7134,NaN
4,ZAF,BCI,AMPLITUD,LTRENDIDX,M,1974-10,101.6797,NaN


In [95]:
CCI_data.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,NLD,CCI,AMPLITUD,LTRENDIDX,M,1973-01,101.2458,NaN
1,NLD,CCI,AMPLITUD,LTRENDIDX,M,1973-02,101.2283,NaN
2,NLD,CCI,AMPLITUD,LTRENDIDX,M,1973-03,101.0854,NaN
3,NLD,CCI,AMPLITUD,LTRENDIDX,M,1973-04,100.8459,NaN
4,NLD,CCI,AMPLITUD,LTRENDIDX,M,1973-05,100.7044,NaN


In [96]:
#remove unnecessary columns
filtered_CCI = CCI_data[['TIME','Value']]
filtered_BCI = BCI_data[['TIME','Value']]
filtered_CLI = CLI_data[['TIME','Value']]
filtered_CCI.head()

,TIME,Value
0,1973-01,101.2458
1,1973-02,101.2283
2,1973-03,101.0854
3,1973-04,100.8459
4,1973-05,100.7044


In [97]:
filtered_BCI.head()

,TIME,Value
0,1974-06,102.0001
1,1974-07,101.8869
2,1974-08,101.7853
3,1974-09,101.7134
4,1974-10,101.6797


In [98]:
#convert to datetime
import datetime
filtered_CCI['TIME']= pd.to_datetime(filtered_CCI['TIME'])
filtered_BCI['TIME']= pd.to_datetime(filtered_BCI['TIME'])
filtered_CLI['TIME']= pd.to_datetime(filtered_CLI['TIME'])
filtered_CCI.head()

C:\Users\beep9\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\beep9\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\beep9\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,TIME,Value
0,1973-01-01,101.2458
1,1973-02-01,101.2283
2,1973-03-01,101.0854
3,1973-04-01,100.8459
4,1973-05-01,100.7044


In [100]:
#rename columns
CCI_df= filtered_CCI.rename(columns={"TIME": "date", "Value": "value"})
CLI_df= filtered_CLI.rename(columns={"TIME": "date", "Value": "value"})
BCI_df= filtered_BCI.rename(columns={"TIME": "date", "Value": "value"})

#further filter data for uniform dates
CCI_df = CCI_df[CCI_df['date'] > '2007-12-31']
BCI_df = BCI_df[BCI_df['date'] > '2007-12-31']
CLI_df = CLI_df[CLI_df['date'] > '2007-12-31']
BCI_df.head()

,date,value
403,2008-01-01,101.5117
404,2008-02-01,101.0985
405,2008-03-01,100.7214
406,2008-04-01,100.4621
407,2008-05-01,100.2753


In [76]:
#for loop function to add filler dates
def date_gen(df):
    
    #create range of days:
    dates = pd.date_range(start ='01/01/2008',end ='11/16/2020')
    
    #create empty dataframe and counter variable
    sample = pd.DataFrame({'date': [], 'value': []})
    counter = 0
    
    #for loop to fill in individual dates
    for date in dates:
        if date.month == df.iloc[0]['date'].month:
            sample = sample.append({'date': date, 'value': df.iloc[counter]['value']},ignore_index=True)
        else:
            counter +=1
            sample = sample.append({'date': date, 'value': df.iloc[counter]['value']},ignore_index=True)
    return sample

In [102]:
#run each set through the for loop function
BCI_adj = date_gen(BCI_df)
CLI_adj = date_gen(CLI_df)
CCI_adj = date_gen(CCI_df)
BCI_adj.head()

,date,value
0,2008-01-01,101.5117
1,2008-01-02,101.5117
2,2008-01-03,101.5117
3,2008-01-04,101.5117
4,2008-01-05,101.5117


In [110]:
#rename value columns to differentatiate values
CCI_adj= CCI_adj.rename(columns={ "value": "CCI"})
CLI_adj= CLI_adj.rename(columns={ "value": "CLI"})
BCI_adj= BCI_adj.rename(columns={"value": "BCI"})

#sanity check
BCI_adj.head(35)

,date,BCI
0,2008-01-01,101.5117
1,2008-01-02,101.5117
2,2008-01-03,101.5117
3,2008-01-04,101.5117
4,2008-01-05,101.5117
5,2008-01-06,101.5117
6,2008-01-07,101.5117
7,2008-01-08,101.5117
8,2008-01-09,101.5117
9,2008-01-10,101.5117


In [111]:
#merge datasets
BCI_CCI = BCI_adj.merge(CLI_adj,how='inner',on='date')
OECD_data = CCI_adj.merge(BCI_CCI, how='inner',on='date')
OECD_data.head()

,date,CCI,BCI,CLI
0,2008-01-01,100.1894,101.5117,100.5212
1,2008-01-02,100.1894,101.5117,100.5212
2,2008-01-03,100.1894,101.5117,100.5212
3,2008-01-04,100.1894,101.5117,100.5212
4,2008-01-05,100.1894,101.5117,100.5212


In [117]:
#convert to csv
sample.to_csv('startdata/OECD_data_2008_to_2020.csv')

In [113]:
# File to Load (Remember to Change These)
stock_data_to_load = "startdata/stocks.csv"

# Read School and Student Data File and store into Pandas DataFrames
stock_data = pd.read_csv(stock_data_to_load)
stock_data.head(10)

,date,ticker,date.1,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume,20-Day SMA,100-Day SMA,Volatility,Lag Gap
0,3/27/2018,GOOGL,3/27/2018,1063.90,1064.54,997.62,1006.94,2940957,0.0,1,1063.90,1064.54,997.62,1006.94,2940957,1006.94,1006.94,66.92,43.66
1,3/26/2018,GOOGL,3/26/2018,1050.60,1059.27,1010.58,1054.09,3272409,0.0,1,1050.60,1059.27,1010.58,1054.09,3272409,1054.09,1054.09,48.69,43.66
2,3/23/2018,GOOGL,3/23/2018,1051.37,1066.78,1024.87,1026.55,2413517,0.0,1,1051.37,1066.78,1024.87,1026.55,2413517,1026.55,1026.55,41.91,-2.72
3,3/22/2018,GOOGL,3/22/2018,1080.01,1083.92,1049.64,1053.15,3418154,0.0,1,1080.01,1083.92,1049.64,1053.15,3418154,1053.15,1053.15,34.28,53.46
4,3/21/2018,GOOGL,3/21/2018,1092.57,1108.70,1087.21,1094.00,1990515,0.0,1,1092.57,1108.70,1087.21,1094.00,1990515,1094.00,1094.00,21.49,39.42
5,3/20/2018,GOOGL,3/20/2018,1098.40,1105.55,1082.42,1095.80,2709310,0.0,1,1098.40,1105.55,1082.42,1095.80,2709310,1095.80,1095.80,23.13,4.40
6,3/19/2018,GOOGL,3/19/2018,1117.76,1119.37,1088.92,1100.07,3076349,0.0,1,1117.76,1119.37,1088.92,1100.07,3076349,1100.07,1100.07,30.45,21.96
7,3/16/2018,GOOGL,3/16/2018,1155.35,1156.81,1131.36,1134.42,2654602,0.0,1,1155.35,1156.81,1131.36,1134.42,2654602,1134.42,1134.42,25.45,55.28
8,3/15/2018,GOOGL,3/15/2018,1149.57,1162.50,1135.66,1150.61,1623868,0.0,1,1149.57,1162.50,1135.66,1150.61,1623868,1150.61,1150.61,26.84,15.15
9,3/14/2018,GOOGL,3/14/2018,1145.80,1159.76,1142.35,1148.89,2033697,0.0,1,1145.80,1159.76,1142.35,1148.89,2033697,1148.89,1148.89,17.41,-4.81


In [ ]:
#Code below to combine OECD data with stock data
OECD_data_to_load = 'startdata/OCED_data_2008_to_2020.csv'
# Read School and Student Data File and store into Pandas DataFrames
OECD_data = pd.read_csv(OECD_data_to_load)

In [88]:
#import API key
from config import api
quandl.ApiConfig.api_key = api
#data = quandl.get_table('WIKI/PRICES', qopts = { 'columns': ['date','ticker','open','high','low','close','volume'] }, ticker = ['AAPL', 'GOOGL','AMZN'], date = { 'gte': '2010-01-01', 'lte': '2020-11-12' })
#df = quandl.get_table('WIKI/PRICES', ticker = ['AAPL', 'GOOGL','AMZN'], date = { 'gte': '2010-01-01', 'lte': '2020-11-12' })
stock_data = quandl.get_table('WIKI/PRICES', ticker = ['GOOGL'], date = { 'gte': '2008-01-01', 'lte': '2018-12-30' })

#convert stock data to datetime
stock_data['date']= pd.to_datetime(stock_data['date'])

#combine stock dataset with OECD information
combine = stock_data.merge(OECD_data,how='inner',on='date')
combine.head(3)

,date,ticker,date.1,open,high,low,close,volume,ex-dividend,split_ratio,...,adj_low,adj_close,adj_volume,20-Day SMA,100-Day SMA,Volatility,Lag Gap,CCI,BLI,CLI
0,2018-03-27,GOOGL,3/27/2018,1063.90,1064.54,997.62,1006.94,2940957,0.0,1,...,997.62,1006.94,2940957,1006.94,1006.94,66.92,43.66,96.29265,96.29265,96.29265
1,2018-03-26,GOOGL,3/26/2018,1050.60,1059.27,1010.58,1054.09,3272409,0.0,1,...,1010.58,1054.09,3272409,1054.09,1054.09,48.69,43.66,96.40020,96.40020,96.40020
2,2018-03-23,GOOGL,3/23/2018,1051.37,1066.78,1024.87,1026.55,2413517,0.0,1,...,1024.87,1026.55,2413517,1026.55,1026.55,41.91,-2.72,97.96307,97.96307,97.96307


In [ ]:

stock_data.nunique()

,date,ticker,date.1,open,high,low,close,volume,ex-dividend,split_ratio,...,adj_low,adj_close,adj_volume,20-Day SMA,100-Day SMA,Volatility,Lag Gap,CCI,BCI,CLI
0,2018-03-27,GOOGL,3/27/2018,1063.90,1064.54,997.62,1006.94,2940957,0.0,1,...,997.62,1006.94,2940957,1006.94,1006.94,66.92,43.66,96.29265,91.12951,97.29847
1,2018-03-27,AMZN,3/27/2018,1572.40,1575.96,1482.32,1497.05,6793279,0.0,1,...,1482.32,1497.05,6793279,1497.05,1497.05,93.64,32.95,96.29265,91.12951,97.29847
2,2018-03-27,AAPL,3/27/2018,173.68,175.15,166.92,168.34,38962839,0.0,1,...,166.92,168.34,38962839,168.34,168.34,8.23,0.27,96.29265,91.12951,97.29847


In [119]:

#data = quandl.get_table('WIKI/PRICES', qopts = { 'columns': ['date','ticker','open','high','low','close','volume'] }, ticker = ['AAPL', 'GOOGL','AMZN'], date = { 'gte': '2010-01-01', 'lte': '2020-11-12' })
#df = quandl.get_table('WIKI/PRICES', ticker = ['AAPL', 'GOOGL','AMZN'], date = { 'gte': '2010-01-01', 'lte': '2020-11-12' })
stock_data = quandl.get_table('WIKI/PRICES', ticker = ['GOOGL'], date = { 'gte': '2008-01-01', 'lte': '2018-12-30' })

In [120]:
stock_data.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
None,,,,,,,,,,,,,,
0,GOOGL,2018-03-27,1063.90,1064.54,997.62,1006.94,2940957.0,0.0,1.0,1063.90,1064.54,997.62,1006.94,2940957.0
1,GOOGL,2018-03-26,1050.60,1059.27,1010.58,1054.09,3272409.0,0.0,1.0,1050.60,1059.27,1010.58,1054.09,3272409.0
2,GOOGL,2018-03-23,1051.37,1066.78,1024.87,1026.55,2413517.0,0.0,1.0,1051.37,1066.78,1024.87,1026.55,2413517.0
3,GOOGL,2018-03-22,1080.01,1083.92,1049.64,1053.15,3418154.0,0.0,1.0,1080.01,1083.92,1049.64,1053.15,3418154.0
4,GOOGL,2018-03-21,1092.57,1108.70,1087.21,1094.00,1990515.0,0.0,1.0,1092.57,1108.70,1087.21,1094.00,1990515.0
